<a href="https://colab.research.google.com/github/siwarbh/Conversational_AI_Chatbot/blob/master/Conversational_AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing


In [ ]:

from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/chatbot_nlp (1).zip', 'r') as zipObj:
   
   zipObj.extractall()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from tensorflow.keras import preprocessing , utils
import os
import yaml
 
dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
tokenizer = preprocessing.text.Tokenizer()
k=tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


{'end': 1,
 'start': 2,
 'you': 3,
 'a': 4,
 'is': 5,
 'i': 6,
 'the': 7,
 'what': 8,
 'are': 9,
 'of': 10,
 'to': 11,
 'do': 12,
 'and': 13,
 'me': 14,
 'not': 15,
 'in': 16,
 'it': 17,
 'tell': 18,
 'that': 19,
 'get': 20,
 'my': 21,
 'when': 22,
 'have': 23,
 'can': 24,
 'joke': 25,
 'am': 26,
 'your': 27,
 'as': 28,
 'like': 29,
 'an': 30,
 'about': 31,
 'how': 32,
 'cross': 33,
 'be': 34,
 'who': 35,
 'for': 36,
 'by': 37,
 'computer': 38,
 'with': 39,
 'no': 40,
 'or': 41,
 "i'm": 42,
 'much': 43,
 'feel': 44,
 'all': 45,
 'any': 46,
 'he': 47,
 'on': 48,
 "don't": 49,
 'was': 50,
 'but': 51,
 'robots': 52,
 'will': 53,
 'favorite': 54,
 'stock': 55,
 'know': 56,
 'think': 57,
 'we': 58,
 'make': 59,
 'die': 60,
 'robot': 61,
 'which': 62,
 'read': 63,
 'could': 64,
 'at': 65,
 'more': 66,
 'hal': 67,
 'from': 68,
 'software': 69,
 'very': 70,
 'should': 71,
 'bad': 72,
 'one': 73,
 'chat': 74,
 'eat': 75,
 'market': 76,
 'did': 77,
 'up': 78,
 'so': 79,
 'said': 80,
 'sense': 81

In [ ]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

 

(564, 22) 22
(564, 60) 60
(564, 60, 1695)


In [ ]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    339000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    339000      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=200, epochs=2000) 
model.save( 'model.h5' ) 


Epoch 1/2000
3/3 [==============================] - 4s 1s/step - loss: 1.1159
Epoch 2/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0571
Epoch 3/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0413
Epoch 4/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0312
Epoch 5/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0227
Epoch 6/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0156
Epoch 7/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0089
Epoch 8/2000
3/3 [==============================] - 4s 1s/step - loss: 1.0032
Epoch 9/2000
3/3 [==============================] - 4s 1s/step - loss: 0.9976
Epoch 10/2000
3/3 [==============================] - 4s 1s/step - loss: 0.9918
Epoch 11/2000
3/3 [==============================] - 4s 1s/step - loss: 0.9842
Epoch 12/2000
3/3 [==============================] - 4s 1s/step - loss: 0.9783
Epoch 13/2000
3/3 [==============================] - 4s 1s/st

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('model.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    339000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    339000      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [ ]:


def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
       if word in tokenizer.word_index:
          tokens_list.append( tokenizer.word_index[ word ] ) 
       if word not in tokenizer.word_index:
           str_to_tokens( input( 'i didn"t understand you ,ask again : ' ))
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(100):
    
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
            
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        
    print( decoded_translation )


KeyboardInterrupt: ignored